In [ ]:
!pip install tiktoken==0.8.0 wandb==0.19.6 weave==0.51.33 docker==7.1.0 openai==1.61.1

In [ ]:
import os
import docker
import re
import tiktoken
import wandb
import weave
from openai import OpenAI

openai_client = OpenAI() # OpenAI gets the API key from the env var
docker_client = docker.from_env()

In [ ]:
# Create containers on network
container_name = "nbtest_wandb"
try:
    container = docker_client.containers.get(container_name)
except docker.errors.NotFound:
    # Ensure network exists
    network_name = "wandb_network"
    try:
        network = docker_client.networks.get(network_name)
    except docker.errors.NotFound:
        network = docker_client.networks.create(network_name)

    # Create container
    container = docker_client.containers.run(
        image="code-runner-client",
        name=container_name,
        shm_size="512mb",
        restart_policy={"Name": "unless-stopped"},
        network=network_name,
        detach=True,
        command=["/bin/sh", "-c", "sleep infinity"],
    )

In [ ]:
## Start: Question / prompt
question = """
""".strip()

In [ ]:
## 1) Generate title
system_prompt = """
You are an AI that generates concise, descriptive titles based on the given question.
Your response must be in the format: <title>Generated title here</title>. "
Keep the title brief and relevant to the question.

Title examples:
Question: "What are the best practices for securing a FastAPI backend?"
<title>What Securing FastAPI best practices</title>

Question: "What are the key considerations when designing an LLM evaluation framework?"
<title>Designing an LLM eval framework</title>

Question: "How do I optimize OpenAI API costs for large-scale applications?"
<title>Optimizing OpenAI API costs</title>
""".strip()

oai_response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": f"Question: {question}"},
    ],
    temperature=0.7
)
completion_response = oai_response.choices[0].message.content

pattern = r'<title>(.*?)</title>'
title = re.findall(pattern, completion_response)
print(title)

In [ ]:
## 2) Generate plan 

In [ ]:
"""
3) Loop
- Generate response 
- Parse actions
- 1. Run code 
- 2. Get output
- Check if complete <answer>
- Check if broken <restart_server></restart_server>
"""

In [ ]:
# async def run_container_command(container):
#     exec_result = container.exec_run(
#         cmd=["python", "-c", "print('Hello, World!')"], stdout=True, stderr=True
#     )
#     return exec_result.output.decode("utf-8")